In [ ]:
from farter import ModelConfig

model_config = ModelConfig.load('nvidia_v1')
print(model_config)


In [ ]:
from farter import CandleDataFetcher

model_config.raw_input[0].get_file_path()

data_fetcher = CandleDataFetcher()
raw_data = data_fetcher.fetch_raw_data(model_config)

In [ ]:
from farter import calculate_indicators

indicators = calculate_indicators(model_config, raw_data)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

nvda = raw_data[0]
rsi = indicators[0]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Candlestick(x=nvda.t,
                open=nvda.o,
                high=nvda.h,
                low=nvda.l,
                close=nvda.c), row=2, col=1)

fig.add_trace(go.Scatter(x=nvda.t, y=rsi.values), row=1, col=1)

fig.show()